[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/koya-jp/AA-google-colab-kohya/blob/master/comfyui_sagemaker_run.ipynb)

#### ***初期設定***

In [ ]:
# 環境の準備　（ターミナルで実行）

# 新しい環境を作る
conda create -y --name sd_env python=3.10.6
conda activate sd_env
conda install -y ipykernel

# 環境をリセット（削除）
conda activate sd_env
conda deactivate
conda remove -y -n sd_env --all

#### ***ComfyUIをインストール***

In [ ]:
!python -V
%cd ~
WORKSPACE = "/home/studio-lab-user/comfyui"

# パッケージのインストール
%conda install -y aria2
!pip install xformers==0.0.20 triton==2.0.0 -U
!pip install mediapipe==0.9.1.0 addict yapf fvcore omegaconf

# リポジトリをクローン （コミットハッシュ → 6d3dee9d16254979592b95399835a54428b3cea6）
# WORKSPACE が存在すれば処理をスキップする。
![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE
%cd $WORKSPACE

# ComfyUI をアップデートする
# !git pull

# 依存関係をインストール
!pip install -r requirements.txt

# custom node のインストール
CUSTOM_NODE_DIR = WORKSPACE + "/custom_nodes"
!git clone -b v1.5 https://github.com/camenduru/comfy_controlnet_preprocessors {CUSTOM_NODE_DIR}/comfy_controlnet_preprocessors
!git clone https://github.com/ltdrdata/ComfyUI-Manager {CUSTOM_NODE_DIR}/ComfyUI-Manager
!git clone https://github.com/space-nuko/ComfyUI-OpenPose-Editor {CUSTOM_NODE_DIR}/ComfyUI-OpenPose-Editor
!git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui {CUSTOM_NODE_DIR}/efficiency-nodes-comfyui
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts {CUSTOM_NODE_DIR}/ComfyUI-Custom-Scripts
!git clone https://github.com/ssitu/ComfyUI_NestedNodeBuilder {CUSTOM_NODE_DIR}/ComfyUI_NestedNodeBuilder
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack {CUSTOM_NODE_DIR}/ComfyUI-Impact-Pack
!git clone https://github.com/twri/sdxl_prompt_styler {CUSTOM_NODE_DIR}/sdxl_prompt_styler
!git clone https://github.com/RockOfFire/ComfyUI_Comfyroll_CustomNodes {CUSTOM_NODE_DIR}/ComfyUI_Comfyroll_CustomNodes
!git clone https://github.com/rgthree/rgthree-comfy {CUSTOM_NODE_DIR}/rgthree-comfy
!git clone https://github.com/failfa-st/failfast-comfyui-extensions {CUSTOM_NODE_DIR}/failfast-comfyui-extensions
!git clone https://github.com/WASasquatch/was-node-suite-comfyui {CUSTOM_NODE_DIR}/was-node-suite-comfyui

# custom node のセットアップ

# comfy_controlnet_preprocessors
%cd {CUSTOM_NODE_DIR}/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts

# ComfyUI-Impact-Pack（Adetailler）
%cd {CUSTOM_NODE_DIR}/ComfyUI-Impact-Pack
!git submodule update --init --recursive
!python install.py
# !sed -i -e "s/dependency_version = 9/dependency_version = 2/" {CUSTOM_NODE_DIR}/ComfyUI-Impact-Pack/impact-pack.ini
!sed -i -e "s/mmdet_skip = True/mmdet_skip = False/" {CUSTOM_NODE_DIR}/ComfyUI-Impact-Pack/impact-pack.ini

# ComfyUI-OpenPose-Editor/js
!sed -i -e "s/const height = 1000/const height = 600/" {CUSTOM_NODE_DIR}/ComfyUI-OpenPose-Editor/js/openpose.js

# was-node-suite-comfyui
!pip install -r {CUSTOM_NODE_DIR}/was-node-suite-comfyui/requirements.txt

# 空き容量を表示
!df -h

#### ***Checkpointをインストール***

In [ ]:
#@markdown ##***SDXL ***
WORKSPACE = "/home/studio-lab-user/comfyui"
LODE_MODE = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"

%cd $WORKSPACE
!rm -rf /home/studio-lab-user/.cache # キャッシュの削除

# base
!rm -fr {WORKSPACE}/models/checkpoints/*
!{LODE_MODE} https://civitai.com/api/download/models/164360 -d {WORKSPACE}/models/checkpoints -o BreakDomainXL-v05g.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/141958 -d {WORKSPACE}/models/checkpoints -o BreakDomainXL-v02g.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/134557 -d {WORKSPACE}/models/checkpoints -o counterfeitxl.safetensors

# refiner
REFINER_URL = "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors" #@param {type:"string"}
!{LODE_MODE} {REFINER_URL} -d {WORKSPACE}/models/checkpoints -o sd_xl_refiner_1.0.safetensors

# vae
VAE_URL = "https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors"
!{LODE_MODE} {VAE_URL} -d {WORKSPACE}/models/vae -o sdxl_vae.vae.safetensors

# embedding
!rm {WORKSPACE}/models/embeddings/put_embeddings_or_textual_inversion_concepts_here
!{LODE_MODE} https://civitai.com/api/download/models/134583 -d {WORKSPACE}/models/embeddings -o negativeXL_D.safetensors # 露出度のネガティブ 0.5 ~ 0.7 くらいで普通。
!{LODE_MODE} https://civitai.com/api/download/models/139645 -d {WORKSPACE}/models/embeddings -o peopleneg.safetensors # 人間のネガティブな特徴を学習。実写用
# !{LODE_MODE} https://civitai.com/api/download/models/133283 -d {WORKSPACE}/models/embeddings -o bad-X.pt # 手などの突然変異防止
# !{LODE_MODE} https://civitai.com/api/download/models/128882 -d {WORKSPACE}/models/embeddings -o SimplepositiveXLv1.safetensors # Simpleなポジティブ要素

# lora
!{LODE_MODE} https://civitai.com/api/download/models/137124 -d {WORKSPACE}/models/loras -o DreamART-Style.safetensors
!{LODE_MODE} https://civitai.com/api/download/models/135867 -d {WORKSPACE}/models/loras -o detail-tweaker-xl.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/121538 -d {WORKSPACE}/models/loras -o watercolor-style.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/136749 -d {WORKSPACE}/models/loras -o ChibiStyleXL.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/138938 -d {WORKSPACE}/models/loras -o Linear-Manga-Style.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/127510 -d {WORKSPACE}/models/loras -o greg_rutkowski_xl_2.safetensors
# !{LODE_MODE} https://civitai.com/api/download/models/118427 -d {WORKSPACE}/models/loras -o perfect-eyes-xl.safetensors

# controlnet
# !{LODE_MODE} https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -d {WORKSPACE}/models/controlnet -o OpenPoseXL2.safetensors
!{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-canny-rank256.safetensors
# !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-depth-rank256.safetensors
# !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-recolor-rank256.safetensors
# !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-sketch-rank256.safetensors

# 確認
%cd {WORKSPACE}/models/
!ls -la checkpoints vae upscale_models embeddings loras controlnet

# 空き容量を表示
!df -h

#### ***画像生成 の結果をZipにする***

In [ ]:
import os
import glob
import datetime
import zipfile
import shutil

# 今日日付を取得
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# 画像のパス
img_path = f"/home/studio-lab-user/comfyui/output"
# ZIPファイルのパス
zip_path = f"/home/studio-lab-user/outputs_{today}.zip"
# 不要なフォルダのパス
delete_path = f"/home/studio-lab-user/Delete/{today}"


# ZIPファイルに圧縮する関数
def zip_files(img_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        # サブフォルダとファイルの全てのパスを取得
        for root, dirs, files in os.walk(img_path):
            for file in files:
                # ZIPファイルに書き込むパスを相対パスにする
                rel_path = os.path.relpath(os.path.join(root, file), img_path)
                # ZIPファイルに書き込む
                zf.write(os.path.join(root, file), rel_path)


# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う関数
def zip_and_move(img_path, zip_path, delete_path):
    # 画像のパスに何もフォルダやファイルがなければスキップする
    if len(os.listdir(img_path)) == 0:
        return
    
    # Zipにする
    zip_files(img_path, zip_path)
    
    # 移動先のディレクトリがなければ作成する
    if not os.path.exists(delete_path):
        os.makedirs(delete_path)

    # ワイルドカードにマッチするパスのリストを取得
    move_list = glob.glob(img_path)

    # 移動を実行
    for file in move_list:
        shutil.move(file, delete_path)

        
# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う
zip_and_move(img_path, zip_path, delete_path)

#### ***⭐ 実行 ----------------------------------------------------------------------------***

In [ ]:
# embeddingsの入力例を表示
import re, os
files = os.listdir('/home/studio-lab-user/comfyui/models/embeddings')
for file in files:
    file = re.sub('\\.(pt|safetensors)$', '', file)
    print('(embedding:{}:0.9)'.format(file))

    
# URLを生成
!pip install pyngrok
%cd /home/studio-lab-user/.ngrok2

from pyngrok import conf, ngrok

ngrok_token="YOUR NGROK TOKEN"
conf.get_default().auth_token = ngrok_token
conf.get_default().monitor_thread = False
ssh_tunnels = ngrok.get_tunnels(conf.get_default())

if len(ssh_tunnels) == 0:
    ssh_tunnel = ngrok.connect(8188)
    print('address：'+ssh_tunnel.public_url)# このURLに接続する
else:
    print('address：'+ssh_tunnels[0].public_url)# このURLに接続する

    
# 実行
%cd "/home/studio-lab-user/comfyui"
!python main.py

#### ***容量の整理***

In [ ]:
# 容量の確認
!df -h
!free -h

In [ ]:
# 容量の整理
!rm -rf /home/studio-lab-user/.ipynb_checkpoints
!rm -rf /home/studio-lab-user/.cache # キャッシュの削除
!ls -la /home/studio-lab-user/

In [ ]:
# model-toolkitで軽量化したモデルを Hugging face にアップロードする
Model_Directory="/home/studio-lab-user/ui/models/VAE" # 対象のディレクトリ
File_Name="vae-ft-mse-840000-ema-pruned.vae-0869.vae.pt" # 対象のファイル

HF_File_Name = "vae_mse_840000_ema_semi.pt" # Hugging face のファイル名
User_Repository ="Hugging face のリポジトリ名"  # Hugging face のリポジトリ名

Upload_File = Model_Directory + "/" + File_Name
Token = "Hugging face トークン" #Hugging face トークン

%cd {Model_Directory}

# アップロード
from huggingface_hub import upload_file
upload_file(path_or_fileobj=Upload_File, path_in_repo=HF_File_Name, repo_id=User_Repository, token=Token)